<a href="https://colab.research.google.com/github/jessieliu1/HLVL-audioanalysis/blob/master/EyeFixationsWithVGGishFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install numpy scipy soundfile resampy six

In [3]:
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [4]:
# Grab the VGGish model
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0   155M      0  0:00:01  0:00:01 --:--:--  155M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   869k      0 --:--:-- --:--:-- --:--:--  869k


In [0]:
!cp models/research/audioset/* .

In [6]:
from google.colab import files
files.upload()

Saving vggish_input_modified.py to vggish_input_modified (1).py


{'vggish_input_modified.py': b'# Copyright 2017 The TensorFlow Authors All Rights Reserved.\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n# ==============================================================================\n\n"""Compute input examples for VGGish from audio waveform."""\n\nimport numpy as np\nimport resampy\nfrom scipy.io import wavfile\n\nimport mel_features\nimport vggish_params\n\n\ndef waveform_to_examples(data, sample_rate,hop_length_seconds=None):\n  """Converts au

In [7]:
import vggish_slim
import vggish_params
import vggish_input_modified
import vggish_postprocess
import numpy as np
from scipy.io import wavfile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import scipy


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:

def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)

  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }

In [0]:
def ProcessWithVGGish(vgg, x, sr,hop_length):
  '''Run the VGGish model, starting with a sound (x) at sample rate
  (sr). Return a whitened version of the embeddings. Sound must be scaled to be
  floats between -1 and +1.'''

  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input_modified.waveform_to_examples(wav_data, sr,hop_length)
  print('Log Mel Spectrogram example: ', input_batch[0])

  [embedding_batch] = sess.run([vgg['embedding']],
                               feed_dict={vgg['features']: input_batch})
  return embedding_batch
  # Postprocess the results to produce whitened quantized embeddings.
  pca_params_path = 'vggish_pca_params.npz'

  pproc = vggish_postprocess.Postprocessor(pca_params_path)
  postprocessed_batch = pproc.postprocess(embedding_batch)
  # print('Postprocessed VGGish embedding: ', postprocessed_batch[0])
  return postprocessed_batch[0]


In [10]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.Session()

vgg = CreateVGGishNetwork()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [11]:

sr, wav_data = wavfile.read("/content/drive/My Drive/Experiments/1.wav")
assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]
print(samples.shape)

(14358528, 2)


In [12]:
values = ProcessWithVGGish(vgg,samples,sr,1)

Log Mel Spectrogram example:  [[-4.60517019 -4.60517019 -4.60517019 ... -4.60517019 -4.60517019
  -4.60517019]
 [-4.60516493 -4.60516466 -4.60516473 ... -4.5913113  -4.56679878
  -4.50297035]
 [ 3.83898025  3.86098965  3.86187834 ...  2.19312893  2.45686938
   2.89196845]
 ...
 [ 8.22712457  8.46715877  8.54160253 ...  8.36054963  8.90810603
   8.84854899]
 [ 7.27135771  7.63229146  8.03926786 ...  7.69950079  8.71524054
   8.60150408]
 [ 8.05478424  7.91861928  7.47251224 ...  7.97652325  8.29346308
   8.64135969]]


In [0]:
df = pd.read_csv("/content/drive/My Drive/Experiments/subsampled_b_fixation_timestamps.csv")

In [0]:
df_final = df

In [0]:
features = []
for number in range(128):
    features.append("feature"+str(number))

In [0]:
df_final = pd.concat([df_final,pd.DataFrame(values,columns=features)],axis=1)

In [17]:
df_final.head()

,start_timestamp,end_timestamp,fixation,feature0,feature1,feature2,feature3,feature4,feature5,feature6,...,feature118,feature119,feature120,feature121,feature122,feature123,feature124,feature125,feature126,feature127
0,0,1,slideimage,0.0,0.000000,0.473913,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.05154,0.195064,0.0,0.000000,0.057337
1,1,2,speaker,0.0,0.090622,0.609878,0.0,0.0,0.0,0.618378,...,0.0,0.0,0.164851,0.0,0.0,0.00000,0.080484,0.0,0.118719,0.133484
2,2,3,slidetext,0.0,0.000000,0.525057,0.0,0.0,0.0,0.230552,...,0.0,0.0,0.000000,0.0,0.0,0.00000,0.258882,0.0,0.195439,0.000000
3,3,4,speaker,0.0,0.000000,0.634766,0.0,0.0,0.0,0.299442,...,0.0,0.0,0.145457,0.0,0.0,0.00000,0.159751,0.0,0.273584,0.000000
4,4,5,slidetext,0.0,0.009878,0.727963,0.0,0.0,0.0,0.590634,...,0.0,0.0,0.176966,0.0,0.0,0.00000,0.378839,0.0,0.677478,0.000000


In [18]:
print(df_final.groupby(['fixation']).size())

fixation
slideimage     28
slidetext     177
speaker        94
dtype: int64


In [0]:
df_final_slideimage = df_final[df_final['fixation'] == 'slideimage']
df_final_slidetext = df_final[df_final['fixation'] == 'slidetext']
df_final_speaker = df_final[df_final['fixation'] == 'speaker']

In [0]:
df_final_slidetext = df_final_slidetext.sample(n=28)
df_final_speaker = df_final_speaker.sample(n=28)

In [0]:
df_final = pd.concat([df_final_slideimage, df_final_slidetext, df_final_speaker])
df_final = df_final.drop(['start_timestamp', 'end_timestamp'],axis = 1)
df_final = df_final.sample(frac=1)

In [22]:
X= df_final[features]
y = df_final["fixation"]
print(X)

     feature0  feature1  feature2  feature3  feature4  feature5  feature6  \
179       0.0  0.000000  0.638802       0.0       0.0       0.0  0.006179   
297       0.0  0.000000  0.303533       0.0       0.0       0.0  0.423875   
228       0.0  0.000000  0.496438       0.0       0.0       0.0  0.703057   
185       0.0  0.024727  0.350624       0.0       0.0       0.0  0.377231   
18        0.0  0.000000  0.591949       0.0       0.0       0.0  0.214699   
268       0.0  0.000000  0.863994       0.0       0.0       0.0  0.386131   
11        0.0  0.130057  0.806913       0.0       0.0       0.0  0.095876   
282       0.0  0.000000  0.393873       0.0       0.0       0.0  0.987548   
294       0.0  0.000000  0.203179       0.0       0.0       0.0  0.679887   
218       0.0  0.000000  0.290178       0.0       0.0       0.0  0.305644   
117       0.0  0.000000  0.665162       0.0       0.0       0.0  0.552771   
194       0.0  0.000000  0.673239       0.0       0.0       0.0  0.000000   

In [0]:
train_X,test_X,train_y,test_y=train_test_split(X,y)

In [0]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

## try logistic regression

In [25]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
rfe = RFE(logreg, 20)
rfe = rfe.fit(X, y)
print(rfe.support_)
print(rfe.ranking_)

[False False False False False False  True False False False False False
 False  True False False False  True False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False  True  True False False
 False  True False False False False  True False False False False False
 False False  True False False False False  True False False False False
 False False False False False False False False False  True  True False
 False  True  True False False False False  True False  True  True  True
 False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False  True False False False]
[109  56  44  69  68  65   1   8  62  73  38  57  77   1  58  51  54   1
  92  18  85  31  89  46  61  26  36  10   1  59  21  43  25  33   6  39
  20  81  79  40  19  83  24  87   1   1  30  37  16   1   5  97  95  93
 

In [26]:
features = np.array(features)
print(features)

['feature0' 'feature1' 'feature2' 'feature3' 'feature4' 'feature5'
 'feature6' 'feature7' 'feature8' 'feature9' 'feature10' 'feature11'
 'feature12' 'feature13' 'feature14' 'feature15' 'feature16' 'feature17'
 'feature18' 'feature19' 'feature20' 'feature21' 'feature22' 'feature23'
 'feature24' 'feature25' 'feature26' 'feature27' 'feature28' 'feature29'
 'feature30' 'feature31' 'feature32' 'feature33' 'feature34' 'feature35'
 'feature36' 'feature37' 'feature38' 'feature39' 'feature40' 'feature41'
 'feature42' 'feature43' 'feature44' 'feature45' 'feature46' 'feature47'
 'feature48' 'feature49' 'feature50' 'feature51' 'feature52' 'feature53'
 'feature54' 'feature55' 'feature56' 'feature57' 'feature58' 'feature59'
 'feature60' 'feature61' 'feature62' 'feature63' 'feature64' 'feature65'
 'feature66' 'feature67' 'feature68' 'feature69' 'feature70' 'feature71'
 'feature72' 'feature73' 'feature74' 'feature75' 'feature76' 'feature77'
 'feature78' 'feature79' 'feature80' 'feature81' 'feature82' 

In [27]:
logistic_features = features[rfe.support_]
print(logistic_features)

['feature6' 'feature13' 'feature17' 'feature28' 'feature44' 'feature45'
 'feature49' 'feature54' 'feature62' 'feature67' 'feature81' 'feature82'
 'feature85' 'feature86' 'feature91' 'feature93' 'feature94' 'feature95'
 'feature113' 'feature124']


## try with all features

In [28]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)
logreg.fit(train_X, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
pred_y = logreg.predict(test_X)

In [30]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_X, test_y)))

Accuracy of logistic regression classifier on test set: 0.48


In [31]:
from sklearn.metrics import confusion_matrix
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[3 2 1]
 [2 4 2]
 [0 4 3]]


## try with selected features

In [32]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logreg.fit(train_X[logistic_features], train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [33]:
pred_y = logreg.predict(test_X[logistic_features])
print(pred_y)

['slidetext' 'speaker' 'slidetext' 'speaker' 'slidetext' 'slidetext'
 'speaker' 'slidetext' 'slidetext' 'slideimage' 'speaker' 'slideimage'
 'slideimage' 'slidetext' 'speaker' 'slidetext' 'speaker' 'slideimage'
 'speaker' 'slidetext' 'speaker']


In [34]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_X[logistic_features], test_y)))

Accuracy of logistic regression classifier on test set: 0.52


In [35]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[3 2 1]
 [1 4 3]
 [0 3 4]]


## try svc with linear kernel

In [36]:
svmLinear = SVC(kernel="linear", C=0.15)
svmLinear.fit(train_X,train_y)

SVC(C=0.15, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
pred_y = svmLinear.predict(test_X)

In [38]:
print('Accuracy of Linear SVM classifier on test set: {:.2f}'.format(svmLinear.score(test_X, test_y)))

Accuracy of Linear SVM classifier on test set: 0.38


In [39]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[6 0 0]
 [7 0 1]
 [5 0 2]]


## try with logit features

In [40]:
svmLinear = SVC(kernel="linear", C=200)
svmLinear.fit(train_X[logistic_features], train_y)

SVC(C=200, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
pred_y = svmLinear.predict(test_X[logistic_features])

In [42]:
print('Accuracy of linear svm classifier on test set: {:.2f}'.format(svmLinear.score(test_X[logistic_features], test_y)))

Accuracy of linear svm classifier on test set: 0.38


In [43]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[2 3 1]
 [2 4 2]
 [2 3 2]]


In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
C_range = np.linspace(180,300, 100)
param_grid = dict(C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svmLinear, param_grid=param_grid, cv=cv)

In [45]:
grid.fit(train_X[logistic_features], train_y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=SVC(C=200, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([180.     , 181.21212, ..., 298.78788, 300.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [46]:
grid.best_estimator_

SVC(C=180.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
pred_y = grid.predict(test_X[logistic_features])

In [48]:
print('Accuracy of linear SVM classifier on test set: {:.2f}'.format(grid.score(test_X[logistic_features], test_y)))

Accuracy of linear SVM classifier on test set: 0.33


In [49]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[1 4 1]
 [2 4 2]
 [2 3 2]]


In [50]:
test_y.index

Int64Index([200, 188, 120,  32, 128, 133, 247, 194,  85,  11, 265, 179, 148,
            297,  28, 101, 193, 150, 117, 261, 243],
           dtype='int64')

In [0]:
#pred_y ==1 and test_y == 1 and pred_y==test_y

## try SVM with rbf kernel

In [0]:
svmrbf = SVC(gamma=2, C=1)

perform grid search 

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
C_range = np.linspace(20,22, 20)
gamma_range = np.linspace(0.2, 0.3, 20)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svmrbf, param_grid=param_grid, cv=cv)

In [55]:
grid.fit(train_X, train_y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'gamma': array([0.2    , 0.20526, 0.21053, 0.21579, 0.22105, 0.22632, 0.23158,
       0.23684, 0.24211, 0.24737, 0.25263, 0.25789, 0.26316, 0.26842,
       0.27368, 0.27895, 0.28421, 0.28947, 0.29474, 0.3    ]), 'C': array([20.     , 20.10526, 20.21053, 20.31579, 20.42105, 20.52632,
       20.63158, 20.73684, 20.84211, 20.94737, 21.05263, 21.15789,
       21.26316, 21.36842, 21.47368, 21.57895, 21.68421, 21.78947,
       21.89474, 22.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, ver

In [56]:
grid.best_estimator_

SVC(C=20.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [0]:
pred_y = grid.predict(test_X)

In [58]:
print('Accuracy of rbf SVM classifier on test set: {:.2f}'.format(grid.score(test_X, test_y)))

Accuracy of rbf SVM classifier on test set: 0.43


In [59]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[3 2 1]
 [1 5 2]
 [3 3 1]]


## try with logit features

In [0]:
svmrbf = SVC()

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
C_range = np.linspace(5, 15, 20)
gamma_range = np.linspace(0, 1, 20)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svmrbf, param_grid=param_grid, cv=cv)

In [62]:
grid.fit(train_X[logistic_features], train_y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'gamma': array([0.     , 0.05263, 0.10526, 0.15789, 0.21053, 0.26316, 0.31579,
       0.36842, 0.42105, 0.47368, 0.52632, 0.57895, 0.63158, 0.68421,
       0.73684, 0.78947, 0.84211, 0.89474, 0.94737, 1.     ]), 'C': array([ 5.     ,  5.52632,  6.05263,  6.57895,  7.10526,  7.63158,
        8.15789,  8.68421,  9.21053,  9.73684, 10.26316, 10.78947,
       11.31579, 11.84211, 12.36842, 12.89474, 13.42105, 13.94737,
       14.47368, 15.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
      

In [63]:
grid.best_estimator_

SVC(C=9.210526315789473, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.3157894736842105,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
pred_y = grid.predict(test_X[logistic_features])

In [65]:
print('Accuracy of linear svm classifier on test set: {:.2f}'.format(grid.score(test_X[logistic_features], test_y)))

Accuracy of linear svm classifier on test set: 0.43


In [66]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[2 3 1]
 [1 5 2]
 [2 3 2]]


## trying out PCA

In [0]:
pca = PCA()

In [68]:
pca.fit(train_X)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [69]:
(pca.explained_variance_ratio_ > 0.007).sum()
# if variance explained is more than 1/128 then the vector is considered

16

In [0]:
pca = PCA(n_components=17)

In [71]:
pca.fit(train_X)

PCA(copy=True, iterated_power='auto', n_components=17, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [72]:
pca.explained_variance_ratio_

array([0.45285683, 0.10266738, 0.08084848, 0.07211124, 0.03973495,
       0.03093003, 0.02805274, 0.0257064 , 0.02054793, 0.01878964,
       0.0145652 , 0.01302614, 0.01227125, 0.010145  , 0.00967882,
       0.00853289, 0.00686901])

In [0]:
train_t_X = pca.transform(train_X)

In [0]:
test_t_X = pca.transform(test_X)

In [75]:
train_t_X.shape

(63, 17)

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
svmLinear = SVC(kernel="linear", C=200)
C_range = np.linspace(850,1250, 10)
param_grid = dict(C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svmLinear, param_grid=param_grid, cv=cv)

In [77]:
grid.fit(train_t_X, train_y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=SVC(C=200, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([ 850.     ,  894.44444,  938.88889,  983.33333, 1027.77778,
       1072.22222, 1116.66667, 1161.11111, 1205.55556, 1250.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [78]:
grid.best_estimator_

SVC(C=850.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
pred_y = grid.predict(test_t_X)

In [80]:
print('Accuracy of linear svm classifier on test set: {:.2f}'.format(grid.score(test_t_X, test_y)))

Accuracy of linear svm classifier on test set: 0.24


In [81]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[2 3 1]
 [3 2 3]
 [2 4 1]]


# trying KNN with PCA

In [0]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [83]:
K_range = np.linspace(1,50, 100,dtype=int)
print(K_range)
param_grid = dict(n_neighbors=K_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(neigh, param_grid=param_grid, cv=cv)

[ 1  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12
 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23 24
 24 25 25 26 26 27 27 28 28 29 29 30 30 31 31 32 32 33 33 34 34 35 35 36
 36 37 37 38 38 39 39 40 40 41 41 42 42 43 43 44 44 45 45 46 46 47 47 48
 48 49 49 50]


In [84]:
grid.fit(train_t_X,train_y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  1, ..., 49, 50])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [85]:
grid.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [0]:
pred_y = grid.predict(test_t_X)

In [87]:
print('Accuracy of linear svm classifier on test set: {:.2f}'.format(grid.score(test_t_X, test_y)))

Accuracy of linear svm classifier on test set: 0.33


In [88]:
confusion_mat = confusion_matrix(test_y, pred_y)
print(confusion_mat)

[[4 2 0]
 [4 2 2]
 [2 4 1]]


## explore the features with incorrect prediction

In [0]:
gesture_vectors = test_X[correct_gestures_index][logistic_features]

In [0]:
gesture_vectors

In [0]:
scipy.spatial.distance.cosine(gesture_vectors.loc[200,:],gesture_vectors.loc[247,:])

In [0]:
average_detected_gesture = gesture_vectors.sum()

In [0]:
scipy.spatial.distance.cosine(gesture_vectors.loc[200,:],average_detected_gesture)

In [0]:
df_8m = pd.DataFrame(columns=["video_id","start_second"]+list(features))

In [0]:
import glob, os
for file in glob.glob("youtube8m/*.wav"):
    sr, wav_data = wavfile.read(file)
    assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
    samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]
    values = ProcessWithVGGish(vgg,samples,sr,1)
    
    temp_df = pd.DataFrame(values,columns=features)
    temp_df['video_id'] = file[10:-4]
    temp_df = temp_df.reset_index().rename(columns={'index':'start_second'})
    df_8m = pd.concat([df_8m,temp_df[df_8m.columns]],axis=0).reset_index(drop=True)

In [0]:
df_8m.tail()

In [0]:
df_8m.shape

In [0]:
df_8m_log = df_8m[logistic_features]

In [0]:
yt8m_rows_selected = []
yt8m_rows_selected_cosine = []

In [0]:
df_8m_log.loc[1,:]

In [0]:
for i in df_8m_log.index:
    cosin_dist = scipy.spatial.distance.cosine(df_8m_log.loc[i,:],average_detected_gesture)
    if cosin_dist >= 0.1:
        yt8m_rows_selected.append(i)
        yt8m_rows_selected_cosine.append(cosin_dist)

In [0]:
yt8m_rows_selected = np.array(yt8m_rows_selected)

In [0]:
yt8m_rows_selected_sorted = yt8m_rows_selected[np.argsort(yt8m_rows_selected_cosine)]

In [0]:
df_8m_top = df_8m.loc[yt8m_rows_selected_sorted,:]

In [0]:
df_8m_top.shape

In [0]:
rank = 1
for i in df_8m_top.index:
    vidId = df_8m_top.loc[i,"video_id"]
    print(vidId)
    start_second = df_8m_top.loc[i,"start_second"]
    a = os.system("ffmpeg -ss {} -t {} -i {} {}".format(start_second,1,"youtube8m/"+vidId+".wav","yt8m_similar/"+"{0:03}".format(rank)+vidId+str(start_second)+".wav"))
    print("{0:03}".format(rank)+vidId+str(start_second)+".wav")
    rank += 1

In [0]:
rank

## find least similar rows

In [0]:
yt8m_rows_selected = []
yt8m_rows_selected_cosine = []

In [0]:
for i in df_8m_log.index:
    cosin_dist = scipy.spatial.distance.cosine(df_8m_log.loc[i,:],average_detected_gesture)
    if cosin_dist <= 0.04:
        yt8m_rows_selected.append(i)
        yt8m_rows_selected_cosine.append(cosin_dist)

In [0]:
yt8m_rows_selected = np.array(yt8m_rows_selected)

In [0]:
np.flip(np.argsort(yt8m_rows_selected_cosine),axis=0)

In [0]:
yt8m_rows_selected_sorted = yt8m_rows_selected[np.flip(np.argsort(yt8m_rows_selected_cosine),axis=0)]

In [0]:
df_8m_top = df_8m.loc[yt8m_rows_selected_sorted,:]

In [0]:
df_8m_top.shape

In [0]:
rank = 1
for i in df_8m_top.index:
    vidId = df_8m_top.loc[i,"video_id"]
    print(vidId)
    start_second = df_8m_top.loc[i,"start_second"]
    a = os.system("ffmpeg -ss {} -t {} -i {} {}".format(start_second,1,"youtube8m/"+vidId+".wav","yt8m_least_similar/"+"{0:03}".format(rank)+vidId+str(start_second)+".wav"))
    print("{0:03}".format(rank)+vidId+str(start_second)+".wav")
    rank += 1

## try to find how scores to on detected and non detected gestures

In [0]:
#frame numbers of questions
framestamps = np.array([1410 ,2160 ,3180 ,3630 ,5730 ,6720 ])

In [0]:
timestamps = framestamps/30

In [0]:
timestamps

In [0]:
test = df_final.iloc[timestamps,:]

In [0]:
train_indexes = ~df_final.index.isin(timestamps)

In [0]:
train = df_final[train_indexes]

In [0]:
train_X,train_y = train[features],train["gesture_detected"]

In [0]:
test_X,test_y = test[features],test["gesture_detected"]

In [0]:
test_X

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
C_range = np.linspace(180,300, 100)
param_grid = dict(C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svmLinear, param_grid=param_grid, cv=cv)

In [0]:
grid.fit(train_X[logistic_features], train_y)

In [0]:
pred_y = grid.predict(test_X[logistic_features])

In [0]:
print('Accuracy of linear SVM classifier on test set: {:.2f}'.format(grid.score(test_X[logistic_features], test_y)))

In [0]:
pred_y==1

## we are able to predict gesture on question 1,2,5
compare the scores for these questions

In [0]:
df_scores = pd.read_excel("scores_of_test.xlsx")

In [0]:
df_bicycle_scores = df_scores.iloc[:,-7:-1] #select the scores for the bicycle questions from the data frame

In [0]:
df_bicycle_scores.describe()

In [0]:
df_pred_corr_ques= df_bicycle_scores.loc[:,pred_y==1]

In [0]:
df_pred_incorr_ques= df_bicycle_scores.loc[:,pred_y!=1]

In [0]:
from scipy import stats

In [0]:
for i in range(3):
    for j in range(3):
        print(i,j)
        tstat = stats.ttest_rel(df_pred_corr_ques.iloc[:,i],df_pred_incorr_ques.iloc[:,j])
        print(f"for {df_pred_corr_ques.columns[i]} and {df_pred_incorr_ques.columns[j]} is {tstat}")